# G201

## Set up

In [1]:
import pandas as pd
from fastcore.test import *
from fastcore.utils import *
import pyspssio

import raine_tools as rn
from data_harmonising.data import *

from src.config import WORKBOOK, SHEETNAME

In [ ]:
INPUT = "../data/raw/G201_Quest_PA.sav"
OUTPUT = "../data/interim/G201_Quest_PA.sav"

In [ ]:
df, meta = read_sav(INPUT, index="ID")

In [3]:
# wb = pd.read_excel(WORKBOOK, sheet_name=SHEETNAME)
# wb['vn_old'] = wb['Dataset'] + '_' + wb['Variable Name']
# wb['vn_new'] = wb['Dataset'] + '_' + wb['Harmonised Variable Name']
# VARS = wb.loc[wb['Dataset'] == 'G201', "Variable Name"].to_list()

## Data exploration

### The strange case of `G201_IF13`

Why does `G201_IF13` exist, especially when there's no corresponding `G201_IC13`?  
Furthermore, the values don't align with the previous IC? variables.

In [4]:
df.loc[:, ["G201_IF13"]].value_counts() 

G201_IF13
0.0          2414
9.0            27
1.0             4
2.0             1
Name: count, dtype: int64

For all other cases of `G201_IF1` to `G201_IF12`, a 0 means that the incident is related to the incident in the previous IF? variable.  
When `G201_IF13` is 0, the majority of cases were N/A, and values range across the board; this doesn't fit with the existing logic.

In [5]:
df.loc[df["G201_IF13"] == 0, "G201_IF12":"G201_IF13"].value_counts()

G201_IF12  G201_IF13
88.0       0.0          2404
1.0        0.0             6
0.0        0.0             2
2.0        0.0             1
999.0      0.0             1
Name: count, dtype: int64

At least when `G201_IF13` is missing (9), the previous value was also missing (999).

In [6]:
df.loc[df["G201_IF13"] == 9, "G201_IF12":"G201_IF13"].value_counts()

G201_IF12  G201_IF13
999.0      9.0          27
Name: count, dtype: int64

Finally, when `G201_IF13` is 1 or 2, the previous values are 0 or 1; this doesn't specifically highlight any errors.

In [7]:
df.loc[df["G201_IF13"].between(1, 2), "G201_IF12":"G201_IF13"].value_counts()

G201_IF12  G201_IF13
1.0        1.0          3
0.0        1.0          1
1.0        2.0          1
Name: count, dtype: int64

`G201_IF13` does not align with the previous values, and there are clear logical discrepancies.  
Given there is no corresponding `G201_IC13` to explain the few instances of a 1 or 2, and the 0s appear to be unreliable, I plan to drop it.

### Checking that IF? and IC? variables are correctly aligned

In [8]:
IF_ROWS = ['G201_IF1', 'G201_IF2', 'G201_IF3', 'G201_IF4', 'G201_IF5', 'G201_IF6',
           'G201_IF7', 'G201_IF8', 'G201_IF9', 'G201_IF10', 'G201_IF11', 'G201_IF12']

IC_ROWS = ['G201_IC1', 'G201_IC2', 'G201_IC3', 'G201_IC4', 'G201_IC5', 'G201_IC6',
           'G201_IC7', 'G201_IC8', 'G201_IC9', 'G201_IC10', 'G201_IC11', 'G201_IC12']

If `G201_IC?` is N/A ("88888"), the corresponding `G201_IF?` should also be N/A (88).  
Else if `G201_IC?` is Missing ("99999"), the corresponding `G201_IF?` should also be Missing (999).    
Otherwise, the corresponding `G201_IF?` should not be N/A or Missing.

In [9]:
for n in range(1, 13): 
    assert (df.loc[df[f"G201_IC{n}"] == "88888", f"G201_IF{n}"] == 88).all()
    assert (df.loc[df[f"G201_IC{n}"] == "99999", f"G201_IF{n}"] == 999).all()
    assert not df.loc[~df[f"G201_IC{n}"].isin(["88888", "99999"]), f"G201_IF{n}"].isin([88, 999]).all()

The reverse of this should also be true, such that if `G201_IF?` is N/A, the corresponding `G201_IC?` value should also be N/A, and so on.

In [10]:
for n in range(1, 13): 
    assert (df.loc[df[f"G201_IF{n}"] == 88, f"G201_IC{n}"] == "88888").all()
    assert (df.loc[df[f"G201_IF{n}"] == 999, f"G201_IC{n}"] == "99999").all()
    assert not df.loc[~df[f"G201_IF{n}"].isin([88, 999]), f"G201_IC{n}"].isin(["88888", "99999"]).all()

If `G201_IF1` is No (0), then remaining `G201_IF?` values should be N/A (88).  
However, a single instance of this was located at ID 10080. 

In [11]:
df.loc[df["G201_IF1"] == 0].filter(regex=r'I(C|F)[1-5]\b')

,G201_IC1,G201_IC2,G201_IC3,G201_IC4,G201_IC5,G201_IF1,G201_IF2,G201_IF3,G201_IF4,G201_IF5
ID,,,,,,,,,,
10080.0,69100,69290,88888,88888,88888,0.0,1.0,88.0,88.0,88.0


For ID 10080, given both conditions are interrelatated (dermatitis), I believe it's likely the participant would have seen the doctor once for both cases.  
Therefore, it makes sense to change IF1 to 1 and IF2 to 0.

In [12]:
idx = [10080]
df.loc[idx, "G201_IF1"] = 1
df.loc[idx, "G201_IF2"] = 0

Participants with IDs 17890, 19270, and 20370 had missing values in place of N/A values, detected during testing process.

In [13]:
idx = [17890, 19270, 20370]
df.loc[idx, "G201_IC1":"G201_IF12"] = df.loc[idx, "G201_IC1":"G201_IF12"].replace({"99999": "88888", 999: 88})

In [14]:
# If any IF or IC value is missing, all others in that row are also missing
for index, row in df.iterrows():
    if row[IF_ROWS].isin([999]).any():
        assert row[IF_ROWS].isin([999]).all(), f"ID {index}: Not all IF values are missing when one is missing."
    if row[IC_ROWS].isin(['99999']).any():
        assert row[IC_ROWS].isin(['99999']).all(), f"ID {index}: Not all IC values are missing when one is missing."

ID 11642: two values for IF and IC that follow a N/A value.
It appears a couple of cells of data were accidentally copied down a line, for IF3, IF4, IC3 and IC4.

ID 23180: one value for IF and IC that follows a N/A value.
Given IF4 and IC4 match IF2 and IC2, and there's a N/A value in-between, it seems likely they've been accidentally copied over.

In [15]:
df.loc[11642, ["G201_IF3", "G201_IF4"]] = 88
df.loc[11642, ["G201_IC3", "G201_IC4"]] = "88888"

df.loc[23180, ["G201_IF4"]] = 88
df.loc[23180, ["G201_IC4"]] = "88888"

In [16]:
# If any IF or IC value is N/A, all following values are also N/A
for index, row in df.iterrows():
    if row[IF_ROWS].isin([88]).any():
        assert row[IF_ROWS].loc[row[IF_ROWS].isin([88]).idxmax():].isin([88]).all(), f"Row {index}: Not all following IF values are N/A."
    if row[IC_ROWS].isin(['88888']).any():
        assert row[IC_ROWS].loc[row[IC_ROWS].isin(['88888']).idxmax():].isin(['88888']).all(), f"Row {index}: Not all following IC values are N/A."

## Create, Delete, and Rename Variables

Create binary variables `G201_IL`, `G201_INJ` and `G201_HO` to harmonise with other datasets.

In [17]:
CREATE = ["G201_IL", # answered yes to one or more IF variables
          "G201_INJ", # answered yes to one or more INF variables
          "G201_HO"] # answered yes to one or more HOH variables

df[CREATE] = None
meta[CREATE] = None

C:\Users\00113294\AppData\Local\Temp\ipykernel_29384\2694142160.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[CREATE] = None
C:\Users\00113294\AppData\Local\Temp\ipykernel_29384\2694142160.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[CREATE] = None
C:\Users\00113294\AppData\Local\Temp\ipykernel_29384\2694142160.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.con

In [18]:
for var in CREATE:
    assert var in df.columns
    assert var in meta.columns

In [19]:
DELETE = ["DOB", # should not be included in the dataset; captured in Longitudinal Data/Gen2_Ages.sav
          "G201_IF13"] # as stated above, appears unreliable

df.drop(DELETE, axis=1, inplace=True)
meta.drop(DELETE, axis=1, inplace=True)

In [20]:
for var in DELETE:
    assert var not in df.columns
    assert var not in meta.columns

In [21]:
RENAME = {} # no vars to rename

df.rename(columns=RENAME, inplace=True)
meta.rename(columns=RENAME, inplace=True)

In [22]:
for original, updated in RENAME.items():
    assert original not in df.columns
    assert original not in meta.columns
    assert updated in df.columns
    assert updated in meta.columns

## Save Interim Data

In [23]:
def output_metadata(metadata: pd.DataFrame):
    "Convert metadata from a DataFrame to a nested dictionary to be compatible with pyspssio"
    RENAME_COLUMNS = {
        "Label": "var_labels",
        "Field Type": "var_formats",
        "Field Width": "var_column_widths",
        # "Decimals": , # could write a fn with regex to determine based on var format / field type
        "Variable Type": "var_measure_levels",
        "Field Values": "var_value_labels"
    }
    metadata = (metadata
                .rename(index=RENAME_COLUMNS)
                .T
                .to_dict())
    # Remove instances where values are NaN for compatibility when saving with pyspssio
    metadata = {key: {k: v for k, v in value.items() if not pd.isnull(v)} for key, value in metadata.items()}
    
    return metadata

In [24]:
df.reset_index(inplace=True)
pyspssio.write_sav(OUTPUT, df, output_metadata(meta))

C:\Users\00113294\AppData\Local\Temp\ipykernel_29384\3029853039.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df.reset_index(inplace=True)


### Compare interim with raw data

In [6]:
df_old, meta_old = read_sav(INPUT, index="ID")
df_new, meta_new = read_sav(OUTPUT, index="ID")

In [7]:
# Ensure all columns were correctly created, deleted, and/or renamed

old, new = rn.diff(df_old.columns, df_new.columns)

assert all([var in old for var in DELETE])
assert all([var in old for var in RENAME.keys()])
assert all([var in new for var in RENAME.values()])
assert all([var in new for var in CREATE])

NameError: name 'DELETE' is not defined

In [27]:
# Compare raw data, not including variables that were created or deleted, and renaming the old variables to match for comparison

old_cols = [col for col in df_old.columns if col not in DELETE]
new_cols = [col for col in df_new.columns if col not in CREATE]
dfx = df_new[new_cols].compare(df_old[old_cols].rename(RENAME))
dfx

G201_IC2        G201_IC3        G201_IC4        G201_IC5        G201_IC6  \
         self  other     self  other     self  other     self  other     self   
135       NaN    NaN      NaN    NaN      NaN    NaN      NaN    NaN      NaN   
856     88888  99999    88888  99999    88888  99999    88888  99999    88888   
1648    88888  99999    88888  99999    88888  99999    88888  99999    88888   
1750      NaN    NaN    88888  00900    88888  05290      NaN    NaN      NaN   
1945      NaN    NaN      NaN    NaN    88888  99999    88888  99999    88888   
2034      NaN    NaN      NaN    NaN    88888  V7200      NaN    NaN      NaN   

             ... G201_IF8        G201_IF9        G201_IF10        G201_IF11  \
      other  ...     self  other     self  other      self  other      self   
135     NaN  ...      NaN    NaN      NaN    NaN       NaN    NaN       NaN   
856   99999  ...     88.0  999.0     88.0  999.0      88.0  999.0      88.0   
1648  99999  ...     88.0  999.0     88.0  999.0      88.0  999.0      88.0   
1750    NaN  ...      NaN    NaN      NaN    NaN       NaN    NaN       NaN   
1945  99999  ...     88.0  999.0     88.0  999.0      88.0  999.0      88.0   
2034    NaN  ...      NaN    NaN      NaN    NaN       NaN    NaN       NaN   

            G201_IF12         
      other      self  other  
135     NaN       NaN    NaN  
856   999.0      88.0  999.0  
1648  999.0      88.0  999.0  
1750    NaN       NaN    NaN  
1945  999.0      88.0  999.0  
2034    NaN       NaN    NaN  

[6 rows x 46 columns]

In [28]:
from pandas import DataFrame
from typing import List

def compare(df1: DataFrame, # updated data
            df2: DataFrame, # original data
            columns: List[str], # columns impact
            idx: List[int], # IDs impacted
            new_value: str|int, # new value
            old_value: str|int # value that was replaced
            ) -> bool: # Output True if all expected changes pass
    "Check that expected changes were correctly implemented."
    dfx = df1.loc[idx, columns].compare(df2.loc[idx, columns], result_names=("new", "old"))
    dfx = dfx.stack(future_stack=True).swaplevel()
    assert (dfx.loc["new"] == new_value).all().all()
    assert (dfx.loc["old"] == old_value).all().all()
    print("All changes correctly updated.")

In [29]:
# Changes should capture each specific changes, meaning column, the IDs that are affected, and the corresponding change

CHANGES = [
    {
        "columns": ["G201_IF1"],
        "idx": [135],
        "old_value": 0,
        "new_value": 1 
    },
    {
        "columns": ["G201_IF2"],
        "idx": [135],
        "old_value": 1,
        "new_value": 0 
    },
    {
        "columns": ["G201_IF3", "G201_IF4"],
        "idx": [1750],
        "old_value": 1,
        "new_value": 88 
    },
    {
        "columns": ["G201_IC3"],
        "idx": [1750],
        "old_value": "00900",
        "new_value": "88888"
    },
    {
        "columns": ["G201_IC4"],
        "idx": [1750],
        "old_value": "05290",
        "new_value": "88888"
    },
    {
        "columns": ["G201_IF4"],
        "idx": [2034],
        "old_value": 1,
        "new_value": 88
    },
    {
        "columns": ["G201_IC4"],
        "idx": [2034],
        "old_value": "V7200",
        "new_value": "88888"
    },
    {
        "columns": ['G201_IC2', 'G201_IC3', 'G201_IC4', 'G201_IC5', 'G201_IC6', 'G201_IC7', 'G201_IC8', 'G201_IC9', 'G201_IC10', 'G201_IC11', 'G201_IC12'],
        "idx": [856, 1648],
        "old_value": "99999",
        "new_value": "88888"
    },
    {
        "columns": ['G201_IF2', 'G201_IF3', 'G201_IF4', 'G201_IF5', 'G201_IF6', 'G201_IF7', 'G201_IF8', 'G201_IF9', 'G201_IF10', 'G201_IF11', 'G201_IF12'],
        "idx": [856, 1648],
        "old_value": 999,
        "new_value": 88
    },
    {
        "columns": ['G201_IC4', 'G201_IC5', 'G201_IC6', 'G201_IC7', 'G201_IC8', 'G201_IC9', 'G201_IC10', 'G201_IC11', 'G201_IC12'],
        "idx": [1945],
        "old_value": "99999",
        "new_value": "88888"
    },
    {
        "columns": ['G201_IF4', 'G201_IF5', 'G201_IF6', 'G201_IF7', 'G201_IF8', 'G201_IF9', 'G201_IF10', 'G201_IF11', 'G201_IF12'],
        "idx": [1945],
        "old_value": 999,
        "new_value": 88
    },
]

In [30]:
for n, change in enumerate(CHANGES, start=1):
    print(f"Change {n}...")
    columns = change["columns"]
    idx = change["idx"]
    new_value = change["new_value"]
    old_value = change["old_value"]
    compare(df_new, df_old, columns, idx, new_value, old_value)

Change 1...
All changes correctly updated.
Change 2...
All changes correctly updated.
Change 3...
All changes correctly updated.
Change 4...
All changes correctly updated.
Change 5...
All changes correctly updated.
Change 6...
All changes correctly updated.
Change 7...
All changes correctly updated.
Change 8...
All changes correctly updated.
Change 9...
All changes correctly updated.
Change 10...
All changes correctly updated.
Change 11...
All changes correctly updated.


### Compare metadata

In [31]:
# Compare metadata, not including variables that were created or deleted, and renaming the old variables to match for comparison

old_cols = [col for col in meta_old.columns if col not in DELETE]
new_cols = [col for col in meta_new.columns if col not in CREATE]
metax = meta_new[new_cols].compare(meta_old[old_cols].rename(RENAME))
metax

Empty DataFrame
Columns: []
Index: []